# udmey 模擬試験 の質問と回答を抽出

In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
# 問題の html をパース
soup = BeautifulSoup(open("./模擬試験3.html"), 'html.parser')

In [ ]:
# 問題部分を取得
questions = soup.findAll(class_=re.compile('^detailed-result-panel--panel-row*'))

In [ ]:
len(questions)

In [ ]:
questions[1]

In [ ]:
# 選択してしまった 間違いの選択肢を抽出する
def find_checked_wrong_options(q):
    checked_option_list = [o.parent.parent for o in q.findAll("input") if ("checked" in o.attrs.keys())]
    checked_wrong_option_list = [o.text for o in checked_option_list if o.text[-11:] == "(Incorrect)" 
                                 or o.text[-5:]=='(不正解)']

    return checked_wrong_option_list

In [ ]:
# 問題の本文を抽出する
def extract_question_body(question):
    q_body = question.findAll(class_=re.compile("mc-quiz-question--question-prompt*"))[0].text

    return q_body

In [ ]:
# 選択した選択肢と解答を抽出する
def extract_answer_body(question):
    a_list = question.findAll(class_=re.compile("mc-quiz-answer--correct--*"))
    # 上記正規表現のクラス指定方法だとネストされた部分を重複して抽出してしまうので間引く
    answers = [a.text for i, a in enumerate(a_list) if i%2 == 0 ]
    checked_wrong_options = find_checked_wrong_options(question)
    
    # 解答と，選択してしまった間違いの選択肢を返す
    return "\n\n".join(answers + checked_wrong_options)    

In [ ]:
# 問題に対する正誤を抽出する
def extract_correctness(question):
    return "正" if (question.findAll('span')[1].text =="正解" or question.findAll('span')[1].text =="Correct")  else "誤"

In [ ]:
# 定義した関数を用いてコンテンツを抽出
contents = [
    {
        "番号": i+1, 
        "正誤": extract_correctness(q),
        "質問": extract_question_body(q),
        "分類": "",
        "解答・回答": extract_answer_body(q),
        "必要な知識": "",
        "備考": ""
    }
    for (i, q) in enumerate(questions[1:])
]

columns = ["番号", "正誤", "質問", "分類", "解答・回答", "必要な知識", "備考"]

pd.DataFrame(contents, columns=columns)

In [ ]:
pd.DataFrame(contents, columns=columns).to_csv("output.csv", encoding='utf-8', index=False)